<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/Chronos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import torch
import warnings
from chronos import ChronosPipeline

warnings.filterwarnings("ignore")

# CARREGAMENTO DOS DADOS
df = pd.read_excel('data.xlsx')
df['Data'] = pd.to_datetime(df['Data'])
df = df.sort_values('Data').set_index('Data')

# Configurações
colunas_alvo = ['INCC- DI_var_mes', 'Taxa_SELIC_avg', 'CUB Global', 'Cub Material',
                'CUB Mão de Obra', 'CUB Desp. Administrativa', 'CUB Equipamento']
contexts = [24, 60, 120]
horizons = [12, 24]
n_windows = 10
NUM_SAMPLES = 20 # Amostragem para cálculo do CRPS probabilístico

# FUNÇÕES DE MÉTRICAS
def calculate_mase(y_true, y_pred, y_train, seasonal_period=12):
    denom = np.mean(np.abs(np.diff(y_train, seasonal_period)))
    if denom == 0: return np.nan
    return np.mean(np.abs(y_true - y_pred)) / denom

def calculate_crps_samples(y_true, samples):
    """Calcula o CRPS empírico via amostras (Monte Carlo)"""
    y_true = np.array(y_true)
    term1 = np.mean(np.abs(samples - y_true), axis=0)
    s_i = samples[:, np.newaxis, :]
    s_j = samples[np.newaxis, :, :]
    term2 = np.mean(np.abs(s_i - s_j), axis=(0, 1))
    return np.mean(term1 - 0.5 * term2)

# INICIALIZAÇÃO DO CHRONOS
print("Inicializando Chronos-T5-Base na GPU...")
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

results_chronos = []

# LOOP DE EXPERIMENTAÇÃO (ROLLING ORIGIN)
for col in colunas_alvo:
    series = df[col].dropna()
    print(f"Executando Chronos: {col}")

    for C in contexts:
        for H in horizons:
            mase_list, crps_list = [], []

            for w in range(n_windows):
                end_idx = len(series) - w
                start_idx = end_idx - (C + H)
                if start_idx < 0: break

                train = series.iloc[start_idx:start_idx+C].values
                test = series.iloc[start_idx+C:end_idx].values

                try:
                    # O Chronos gera a distribuição futura baseada no contexto C
                    context_tensor = torch.tensor(train)
                    forecast_samples = pipeline.predict(
                        context_tensor,
                        prediction_length=H,
                        num_samples=NUM_SAMPLES,
                    ).numpy()[0]

                    # Mediana das amostras para métrica de ponto
                    forecast_median = np.median(forecast_samples, axis=0)

                    mase_list.append(calculate_mase(test, forecast_median, train))
                    crps_list.append(calculate_crps_samples(test, forecast_samples))
                except Exception as e:
                    continue

            if mase_list:
                results_chronos.append({
                    'Variável': col, 'C': C, 'H': H,
                    'MASE (Mediana)': round(np.median(mase_list), 4),
                    'CRPS (Mediana)': round(np.median(crps_list), 4),
                    'Modelo': 'Chronos-Base'
                })

# RESULTADOS FINAIS
results_df_chronos = pd.DataFrame(results_chronos)
print("\n" + "="*40)
print("RESULTADOS FINAIS - CHRONOS")
print("="*40)
print(results_df_chronos.to_string(index=False))

# Salvar resultados em csv
results_df_chronos.to_csv('RESULTADOS_Chronos.csv', index=False)